# Name: Bhavesh Kewalramani
# Roll No.: A-25
# Section: A
# Semester: VII
# Shift: I
# Batch: A1

## Suppose that the data mining task is to cluster points (with (x,y) representing location) into three clusters. Consider the data points- A1(2,10), A2(2,5), A3(8,4), B1(5,8), B2(7,5), B3(6,4), C1(1,2), C2(4,0). Use the Manhattan distance function. Initially, assume A1, B1, and C1 ascluster centers. Implement the K-means algorithm to show the finalthree clusters.

In [98]:
import numpy as np
import math
from time import time

def manhattan(XA, XB):
    XA = np.array(XA)
    XB = np.array(XB)
    return np.sum(np.abs(XA - XB))

VALID_DISTANCE_ARG = {
  "manhattan": manhattan,
}

class KMeans():
    def __init__(self, k=3, distance="manhattan"):
        self.k = k
        if distance in VALID_DISTANCE_ARG.keys():
            self.distance = VALID_DISTANCE_ARG[distance]
        else:
            raise Exception("distance is not valid")

    def train(self, X,centroids ,max_iteration = 100, tolerance = 0.001, verbose=False):
        start_time = time()
        X = np.array(X)
        
        if len(X.shape) != 2:
            raise Exception("Data must be 2D array")
        
        self.n_features = X[0].shape[0]

        self.centroids = centroids
  
        if verbose:
            print("initial centroid", self.centroids)

        self.cluster_members = [[] for _ in range(self.k)]
    
        iteration = 0
        total_diff = float("inf")
        while iteration < max_iteration:
            if verbose:
                print("iteration", iteration)
                print("centroid", self.centroids)
                
            current_inertia = float(0.0)
            current_cluster_members = [[] for _ in range(self.k)]
            for data_point in X:
                min_distance = float("inf")
                cluster = 0
                for cluster_idx, centroid_i in enumerate(self.centroids):
                    distance = self.distance(centroid_i, data_point)
                    # print("centroid, distance", centroid_i, distance)
                    if distance <= min_distance:
                        cluster = cluster_idx
                        min_distance = distance

                current_cluster_members[cluster].append(data_point)
                current_inertia = current_inertia + min_distance

            if verbose:
                print("cluster member")
                for idx, ccm in enumerate(current_cluster_members):
                    print("cluster" + str(idx), ccm)

            new_centroids = [[] for _ in range(self.k)]
            for cluster_i in range(self.k):
                new_centroid_i = np.zeros(self.n_features)
                members_of_current_cluster = current_cluster_members[cluster_i]
                if len(members_of_current_cluster) > 0:
                    for member in current_cluster_members[cluster_i]:
                        new_centroid_i = new_centroid_i + member
                    new_centroid_i = new_centroid_i / len(members_of_current_cluster) # Get average point from all members
                else:
                    new_centroid_i = self.choose_random_point(X)

                new_centroids[cluster_i] = new_centroid_i

            if verbose:
                print("new centroid", new_centroids)
              
            total_diff = float(0.0)
            for cluster_i in range(self.k):
                total_diff = total_diff + self.distance(self.centroids[cluster_i], new_centroids[cluster_i])

            self.centroids = new_centroids
            self.cluster_members = current_cluster_members
            self.inertia = current_inertia

            if verbose:
                print("total diffs:", total_diff)
                print()

            if total_diff <= tolerance:
                break
            iteration = iteration + 1

        if verbose:
            print(self.cluster_members)
            for idx, cm in enumerate(self.cluster_members):
                print("cluster"+ str(idx), cm)
        print("Training time", (time() - start_time) * 100 , "ms")
        print("Stopped at iteration", iteration+1)
        self.n_iteration = iteration
        return self.predict(X)


    def predict(self, X):
        result = []
        for data_point in X:
            min_distance = float("inf")
            cluster = None
            for cluster_idx, centroid_i in enumerate(self.centroids):
                distance = self.distance(centroid_i, data_point)
                if distance <= min_distance:
                    cluster = cluster_idx
                    min_distance = distance
            result.append(cluster)
        return result

In [117]:
points = [[2, 10],[2, 5],[8, 4],[5, 8],[7, 5],[6, 4],[1, 2],[4, 0]]
centroids = [[2,10],[5,8],[1,2]]

k=3
model_euclidean = KMeans(k)
clusters = model_euclidean.train(points,centroids)

cluster={1:[],2:[],3:[]}
for i in range(len(clusters)):
    cluster[clusters[i]+1].append(points[i])
    print
    
for i in range(k):
    print("Cluster - ",i+1," : ",cluster[i+1])

Training time 0.0995635986328125 ms
Stopped at iteration 2
Cluster -  1  :  [[2, 10]]
Cluster -  2  :  [[8, 4], [5, 8], [7, 5], [6, 4]]
Cluster -  3  :  [[2, 5], [1, 2], [4, 0]]
